# Query application

Python API to query Vespa applications.

This tutorial goes through how to connect to a pre-existing Vespa instance and use the Query API, using the https://cord19.vespa.ai/ app as an example. You can run this tutorial in Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/query.ipynb)

In [25]:
from vespa.application import Vespa

app = Vespa(url = "https://api.cord19.vespa.ai")

## Specify the request body

Full flexibility by specifying the entire request body,
see the [Vespa query language](https://docs.vespa.ai/en/reference/query-api-reference.html).

In [26]:
body = {
  'yql': 'select cord_uid, title, abstract from sources * where userQuery();',
  'hits': 5,
  'query': 'Is remdesivir an effective treatment for COVID-19?',
  'type': 'any',
  'ranking': 'bm25'
}

In [27]:
results = app.query(body=body)

In [28]:
results.number_documents_retrieved

10613

Number of documents returned:

In [29]:
len(results.hits)

5

We can then retrieve specific information from the hit list thorugh the `results.hits` or access the entire Vespa response through `results.json`.

In [30]:
[hit["fields"]["cord_uid"] for hit in results.hits]

['2lwzhqer', '8n6eybze', '8n6eybze', 'xej338lo', '8art2tyj']

## Specify a query model

### Query + term-matching + rank profile

In [31]:
from vespa.query import QueryModel, OR, RankProfile

results = app.query(
    query="Is remdesivir an effective treatment for COVID-19?", 
    query_model = QueryModel(
        match_phase=OR(), 
        rank_profile=RankProfile(name="bm25")
    )
)

In [32]:
results.number_documents_retrieved

268740

### Query + term-matching + ann operator + rank_profile

In [33]:
from vespa.query import QueryModel, QueryRankingFeature, ANN, WeakAnd, Union, RankProfile
from random import random

match_phase = Union(
    WeakAnd(hits = 10), 
    ANN(
        doc_vector="title_embedding", 
        query_vector="title_vector", 
        hits = 10,
        label="title"
    )
)
rank_profile = RankProfile(name="bm25", list_features=True)
query_model = QueryModel(
    query_properties=[QueryRankingFeature(
        name="title_vector", 
        mapping=lambda x: [random() for x in range(768)]
    )],
    match_phase=match_phase, rank_profile=rank_profile
)

In [34]:
results = app.query(query="Is remdesivir an effective treatment for COVID-19?", 
          query_model=query_model)

In [35]:
results.number_documents_retrieved

1579

## Recall specific documents

Let's take a look at the top 3 ids from the last query.

In [36]:
top_ids = [hit["fields"]["id"] for hit in results.hits[0:3]]
top_ids

[269386, 144384, 144385]

Assume that we now want to retrieve the second and third ids above. We can do so with the `recall` argument.

In [37]:
results_with_recall = app.query(query="Is remdesivir an effective treatment for COVID-19?", 
                    query_model=query_model,
                    recall = ("id", top_ids[1:3]))

It will only retrieve the documents with Vespa field `id` that is defined on the list that is inside the tuple.

In [38]:
id_recalled = [hit["fields"]["id"] for hit in results_with_recall.hits]
id_recalled

[144384, 144385]